In [2]:
from mxnet import nd
from mxnet.gluon import nn

class MLP(nn.Block):
    # 声明带有模型参数的层，这里我们声明了两个全链接层。
    def __init__(self, **kwargs):
        # 调用 MLP 父类 Block 的构造函数来进行必要的初始化。这样在构造实例时还可以指定
        super().__init__(**kwargs)
        self.hidden = nn.Dense(256, activation='relu')  # 隐藏层。
        self.output = nn.Dense(10)  # 输出层。

    # 定义模型的前向计算，即如何根据输入计算输出。
    def forward(self, x):
        return self.output(self.hidden(x))

In [3]:
net = MLP()

任何事物都有一个从创建，被使用，再到消亡的过程，在程序语言面向对象编程模型中，对象也有相似的命运：创建、初始化、使用、垃圾回收，不同的阶段由不同的方法负责执行。

## `__init__` 方法

负责对象的初始化，系统执行该方法前，其实该对象已经存在了，要不然初始化什么东西呢？先看例子

In [9]:
# class A(object): python2 必须显示地继承object
class A:
    def __init__(self):
        print("__init__ ")
        super().__init__()    # 或者写作 super(A, self).__init__()

    def __new__(cls):
        print("__new__ ")
        return super().__new__(cls)

    def __call__(self):  # 可以定义任意参数
        print('__call__ ')


A()

__new__ 
__init__ 


从输出结果来看， `__new__` 方法先被调用，返回一个实例对象，接着 `__init__` 被调用。 `__call__` 方法并没有被调用，这个我们放到最后说，先来说说前面两个方法，稍微改写成：

In [12]:
# class A(object): python2 必须显示地继承object
class A:
    def __init__(self):
        print("__init__ ")
        print(self)
        super().__init__()    # 或者写作 super(A, self).__init__()

    def __new__(cls):
        print("__new__ ")
        self = super().__init__(cls)
        print(self)
        return self

    def __call__(self):  # 可以定义任意参数
        print('__call__ ')


A()

__new__ 
None


从输出结果来看，`__new__` 方法的返回值就是类的实例对象，这个实例对象会传递给 `__init__` 方法中定义的 `self` 参数，以便实例对象可以被正确地初始化。

如果 `__new__` 方法不返回值（或者说返回 `None`）那么 `__init__` 将不会得到调用，这个也说得通，因为实例对象都没创建出来，调用 `init` 也没什么意义，此外，Python 还规定，`__init__` 只能返回 `None` 值，否则报错。

`__init__` 方法可以用来做一些初始化工作，比如给实例对象的状态进行初始化：

In [19]:
# class A(object): python2 必须显示地继承object
class A:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        super().__init__()
        
    def __new__(cls):
        print("__new__ ")
        self = super().__init__(cls)
        print(self)
        return self

    def __call__(self):  # 可以定义任意参数
        print('__call__ ')


A()

__new__ 
None


另外，`__init__` 方法中除了 `self` 之外定义的参数，都将与 `__new__` 方法中除 `cls` 参数之外的参数是必须保持一致或者等效。

In [15]:
class B:
    def __init__(self, *args, **kwargs):
        print("init", args, kwargs)

    def __new__(cls, *args, **kwargs):
        print("new", args, kwargs)
        return super().__new__(cls)

B(1, 2, 3)

new (1, 2, 3) {}
init (1, 2, 3) {}


##  `__new__` 方法

一般我们不会去重写该方法，除非你确切知道怎么做，什么时候你会去关心它呢，它作为构造函数用于创建对象，是一个工厂函数，专用于生产实例对象。著名的设计模式之一，单例模式，就可以通过此方法来实现。在自己写框架级的代码时，可能你会用到它，我们也可以从开源代码中找到它的应用场景，例如微型 Web 框架 Bootle 就用到了。参考：https://github.com/bottlepy/bottle/blob/release-0.6/bottle.py

## `__call__` 方法

关于 `__call__` 方法，不得不先提到一个概念，就是可调用对象（callable），我们平时自定义的函数、内置函数和类都属于可调用对象，但凡是可以把一对括号 `()` 应用到某个对象身上都可称之为可调用对象，判断对象是否为可调用对象可以用函数 `callable`

如果在类中实现了 `__call__` 方法，那么实例对象也将成为一个可调用对象，我们回到最开始的那个例子：

In [24]:
a = A
print(callable(a))  # True

True


In [26]:
class Counter:
    def __init__(self, func):
        self.func = func
        self.count = 0

    def __call__(self, *args, **kwargs):
        self.count += 1
        return self.func(*args, **kwargs)

@Counter
def foo():
    pass

for i in range(10):
    foo()

print(foo.count)  # 10

10


我们把 Person 类变成一个可调用对象：

In [29]:
class Person(object):
    def __init__(self, name, gender):
        self.name = name
        self.gender = gender

    def __call__(self, friend):
        print('My name is %s...' % self.name)
        print('My friend is %s...' % friend)

现在可以对 Person 实例直接调用：

In [32]:
p = Person('Bob', 'male')
p('Tim')

My name is Bob...
My friend is Tim...


单看 `p('Tim')` 你无法确定 `p` 是一个函数还是一个类实例，所以，在 Python 中，函数也是对象，对象和函数的区别并不显著。

斐波那契数列:

In [28]:
class Fib(object):
    def __init__(self):
        pass

    def __call__(self, num):
        a, b = 0, 1
        self.l = []

        for i in range(num):
            self.l.append(a)
            a, b = b, a+b
        return self.l

    def __str__(self):
        return str(self.l)
    __rept__ = __str__


f = Fib()
print(f(10))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
